In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from sklearn.neighbors import KDTree
import folium
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#https://github.com/gboeing/osmnx-examples/blob/master/notebooks/05-example-save-load-networks-shapes.ipynb

# from some place name, create a GeoDataFrame containing the geometry of the place
# define a list of place names
place = ['wendake,canada', 
               'Lévis, canada',
               'Québec city, canada',
               "L'ancienne-lorette, canada"
               ]


G = ox.graph_from_place(place, network_type='drive',
                       simplify=False,
                       retain_all=True,
                       buffer_dist = 10000    # pour pas avoir des gens pas mappables?
 )

ox.save_graph_shapefile(G, filename='original-network-shape')

In [3]:
# copy and  speed limits, travel time  simplify and  DONTproject

#https://github.com/gboeing/osmnx-examples/blob/master/notebooks/04-example-simplify-network.ipynb
# simplify the network
G_nobridge = G.copy()


G_nobridge = ox.simplify_graph(G_nobridge)
# add speed limit and travel time after simplifying
#thisworks 
for u, v, k, data in G_nobridge.edges(data=True, keys=True):

  if 'highway' in data and data["highway"] == "motorway":
    data["speed_limit_kmh"] = 100
  elif 'highway':
    data["speed_limit_kmh"] = 50
    
    
for u, v, k, data in G_nobridge.edges(data=True, keys=True): # u et v sont les nodes origines destination
    data['time_minute'] = data['length'] / 1000 * 60 / data['speed_limit_kmh'] 
    

#https://medium.com/@bobhaffner/osmnx-intro-and-routing-1fd744ba23d8
ox.save_graph_shapefile(G_nobridge, filename='G_nobridge')

In [4]:
# copy and add bridge, speed limits, travel time  simplify and project

#https://github.com/gboeing/osmnx-examples/blob/master/notebooks/04-example-simplify-network.ipynb
# simplify the network
G_bridge = G.copy()

# lien vers autoroute duplessis et de la capitale
G_bridge.add_node(999999999999999999999, x= -71.153, y= 46.866)

#longueur calculées dans qgis avec  http://www.qgistutorials.com/en/docs/calculating_line_lengths.html
# à investiguer: adD_edge_lengths..

#québec:
#arrivées
#180699309     nord
#130153234  ouest
#130152949 est

#départs
#180672314   nord
#180688633 ouest
#269683680 est 

G_bridge.add_edge(999999999999999999999, 180699309, key=0,  highway = "motorway", oneway = False, length = 841)
G_bridge.add_edge(999999999999999999999, 130153234, key=0,  highway = "motorway", oneway = False, length = 875)
G_bridge.add_edge(999999999999999999999, 130152949, key=0,  highway = "motorway", oneway = False, length = 1069)
G_bridge.add_edge(180672314,999999999999999999999,  key=0,  highway = "motorway", oneway = False, length = 778)
G_bridge.add_edge(180688633,999999999999999999999,  key=0,  highway = "motorway", oneway = False, length = 1060)
G_bridge.add_edge(269683680,999999999999999999999,  key=0,  highway = "motorway", oneway = False, length = 1196)
#lien vers auroute 20

G_bridge.add_node(88888888888888888888, x= -71.1024, y= 46.8173)


#lallemand
#arrivée
#5728701577 est
#486128981 ouest
#départ
#486135896 ouest
#281651410 est

G_bridge.add_edge(88888888888888888888, 5728701577, key=0,  highway = "motorway", oneway = False, length = 624)
G_bridge.add_edge(88888888888888888888, 486128981,  key=0,  highway = "motorway", oneway = False, length = 1077)
G_bridge.add_edge(486135896, 88888888888888888888 ,  key=0,  highway = "motorway", oneway = False, length = 1050)
G.add_edge(281651410, 88888888888888888888 ,  key=0,  highway = "motorway", oneway = False, length = 782)

# lien entre les deux liens
G_bridge.add_edge(88888888888888888888, 999999999999999999999, key=0, highway = "motorway", oneway = False, length = 6648)
G_bridge.add_edge(999999999999999999999, 88888888888888888888, key=0, highway = "motorway", oneway = False, length = 6648)

G_bridge = ox.simplify_graph(G_bridge)
# add speed limit and travel time after simplifying
#thisworks 
for u, v, k, data in G_bridge.edges(data=True, keys=True):

  if 'highway' in data and data["highway"] == "motorway":
    data["speed_limit_kmh"] = 100
  elif 'highway':
    data["speed_limit_kmh"] = 50
    
    
for u, v, k, data in G_bridge.edges(data=True, keys=True): # u et v sont les nodes origines destination
    data['time_minute'] = data['length'] / 1000 * 60 / data['speed_limit_kmh'] 
    

G_bridge = ox.project_graph(G_bridge)
ox.save_graph_shapefile(G_bridge, filename='G_bridge')

In [5]:
library = ox.geocode("487 rue des trappistines, lévis,canada")
museum = ox.geocode("beauport, Québec, Canada")

In [6]:
nodes_nobridge, edges_nobridge  = ox.graph_to_gdfs(G_nobridge)


In [7]:
#edges_bridge
#nodes_nobridge

In [14]:

tree = KDTree(nodes_nobridge[['y', 'x']], metric='euclidean')
#lib_idx = tree.query([library], k=1, return_distance=False)[0]
#museum_idx = tree.query([museum], k=1, return_distance=False)[0]
#losest_node_to_lib = nodes_nobridge.iloc[lib_idx].index.values[0]
#closest_node_to_museum = nodes_nobridge.iloc[museum_idx].index.values[0]

In [9]:
# 88888888888888888888 jusqu'à rive nord fonctionne
# 88888888888888888888 jusqu'à rive-sud fonctionne
# rive sud jusqu 88888888888888888888 marche pas
# nive nord jusqu'a 88888888888888888888 marche pas

#test = (library[0], library[0])
#test

In [10]:
#readr::write_csv( trajets2 %>% st_set_geometry(NULL) %>% select(-origine, -destination) , "trajets2.csv")
trajets2 = pd.read_csv("trajets2.csv")

# https://stackoverflow.com/questions/16031056/how-to-form-tuple-column-from-two-columns-in-pandas

origine= list(zip(trajets2.origine_y.ravel(),trajets2.origine_x.ravel() ))
destination= list(zip(trajets2.destination_y.ravel(),trajets2.destination_x.ravel() ))
trajets2['origine'] = origine
trajets2['destination'] = destination

In [11]:
#trajets2 = trajets2[ (trajets2["NOM_quartier_origine"]== "Lévis") & (trajets2["NOM_quartier_destination"] == "Montcalm")]
trajets2 =  trajets2[ (trajets2["ID_quartier_origine"] <  50) & (trajets2["ID_quartier_destination"] < 50)]

In [12]:
#trajets2

In [15]:
# http://book.pythontips.com/en/latest/map_filter.html
closest_node_to_origine = list(map(lambda x: nodes_nobridge.iloc[tree.query([x], k=1, return_distance=False)[0]].index.values[0],  
                                  trajets2["origine"]))
closest_node_to_destination = list(map(lambda x: nodes_nobridge.iloc[tree.query([x], k=1, return_distance=False)[0]].index.values[0], 
                                      trajets2["destination"]))


In [16]:
trajets2["closest_node_to_origine"] = closest_node_to_origine
trajets2["closest_node_to_destination"] = closest_node_to_destination
trajets2

,TRIP_ID,origine_x,origine_y,destination_x,destination_y,ID_quartier_origine,ID_quartier_destination,NOM_quartier_origine,NOM_quartier_destination,origine,destination,closest_node_to_origine,closest_node_to_destination
0,1,-71.181039,46.788518,-71.265626,46.734751,37.0,40.0,Lévis,Saint-Romuald,"(46.788517999999996, -71.181039)","(46.734751, -71.265626)",2509788208,2496754005
1,2,-71.287948,46.793157,-71.282687,46.829251,3.0,34.0,Cité Universitaire,Vanier,"(46.793157, -71.287948)","(46.829251, -71.282687)",280618676,3524201637
2,3,-71.267908,46.872975,-71.298406,46.787753,21.0,3.0,Jésuites,Cité Universitaire,"(46.872975, -71.26790799999999)","(46.787753, -71.298406)",303684376,1698551117
3,4,-71.266550,46.873682,-71.285824,46.799252,21.0,6.0,Jésuites,Duberger-Les Saules,"(46.873682, -71.26655)","(46.799252, -71.285824)",303684086,2264015729
4,5,-71.370312,46.811021,-71.269020,46.835109,48.0,14.0,L'Ancienne-Lorette,Neufchâtel-Est/Lebourgneuf,"(46.811021000000004, -71.370312)","(46.835109, -71.26902)",249434457,180665198
5,6,-71.269526,46.842643,-71.277325,46.871153,23.0,19.0,Quartier 4-6,Quartier 4-2,"(46.842643, -71.269526)","(46.871153, -71.277325)",269455671,275074592
6,7,-71.385159,46.834102,-71.295427,46.688628,33.0,44.0,Val-Bélair,Saint-Rédempteur,"(46.834102, -71.385159)","(46.688628, -71.295427)",403654321,1574674603
7,8,-71.190381,46.866764,-71.234965,46.852889,27.0,21.0,Quartier 5-4,Jésuites,"(46.866764, -71.190381)","(46.852889000000005, -71.234965)",279225385,265024270
8,9,-71.311688,46.804388,-71.376266,46.827669,6.0,7.0,Duberger-Les Saules,L'Aéroport,"(46.804388, -71.311688)","(46.827669, -71.376266)",292944074,249486884
9,10,-71.201136,46.745676,-71.294660,46.806515,40.0,6.0,Saint-Romuald,Duberger-Les Saules,"(46.745676, -71.201136)","(46.806515000000005, -71.29466)",449007429,180713221


In [ ]:

#trajets2[
#trajets2[trajets2["closest_node_to_destination"]== 265685900]
#tannant = pouet[pouet["destination"]]

#nx.shortest_path(G_nobridge,265685900,3388629610, )
#nodes_nobridge[nodes_nobridge["osmid"] == 265685900 ]
#nodes_nobridge[nodes_nobridge["osmid"] == 3388629610 ]


#fig, ax = ox.plot_graph(G)
#ax.scatter(library[1], library[0], c='red', s=100)
#ax.scatter(museum[1], museum[0], c='blue', s=100)
#plt.show()


In [ ]:
#fig, ax = ox.plot_graph_route(G, route, fig_height=10, 
#                              fig_width=10, 
#                              show=False, close=False, 
#                              edge_color='black',
#                              orig_dest_node_color='green',
#                              route_color='green')
#ax.scatter(library[1], library[0], c='red', s=100)
#ax.scatter(museum[1], museum[0], c='blue', s=100)
#plt.show()

In [63]:
def tryconvert_nobridge(closest_node_to_origine, closest_node_to_destination):
  try:
    goodpath = nx.shortest_path(G_nobridge,
                   closest_node_to_origine,
                   closest_node_to_destination, weight = "time_minute")
    return(goodpath)
  except:
    return([0])

def tryconvert_bridge(closest_node_to_origine, closest_node_to_destination):
  try:
    goodpath = nx.shortest_path(G_bridge,
                   closest_node_to_origine,
                   closest_node_to_destination, weight = "time_minute")
    return(goodpath)
  except:
    return( [0])


#routesss_nobridge =  list(map(tryconvert,closest_node_to_origine, closest_node_to_destination))



tryconvert_bridge(307561537,2778622453) # 0
#type(tryconvert_bridge(2509788208, 2496754005))


[0]

In [64]:
routesss_nobridge =  list(map(tryconvert_nobridge, 
                              closest_node_to_origine, # closest_node_to_origine[1:4]
                              closest_node_to_destination)) # closest_node_to_destination[1:4]

In [65]:
routesss_bridge =  list(map(tryconvert_bridge, 
                              closest_node_to_origine, # closest_node_to_origine[1:4]
                              closest_node_to_destination)) # closest_node_to_destination[1:4]

In [51]:
#routesss_nobridge

In [ ]:
#routesss_bridge =  list(map(lambda x,y : 
#                     nx.shortest_path(G_bridge,
#                                  x, 
#                                  y, weight = "time_minute"), 
#                    closest_node_to_origine,
#                    closest_node_to_destination))

In [29]:
# NetworkXNoPath: No path to 482801757. -- get first** row with this destination
fatigant = trajets2[trajets2["closest_node_to_destination"]==2778622453].iloc[0]
nx.shortest_path(G_nobridge,fatigant["closest_node_to_origine"],fatigant["closest_node_to_destination"], weight = "time_minute")

#fig, ax = ox.plot_graph(G,  fig_height=10, 
#                              fig_width=10, 
#                              show=False, close=False, 
#                              edge_color='black')
#ax.scatter(fatigant["origine_x"], fatigant["origine_y"], c='red', s=100)
#ax.scatter(fatigant["destination_x"], fatigant["destination_y"], c='blue', s=100)
#plt.show()


NetworkXNoPath: No path to 2778622453.

In [ ]:
# NetworkXNoPath: No path to 482801757. -- get first row with this destination
fatigant = trajets2[trajets2["closest_node_to_destination"]==482801757].iloc[1]

fig, ax = ox.plot_graph(G,  fig_height=10, 
                              fig_width=10, 
                              show=False, close=False, 
                              edge_color='black')
ax.scatter(fatigant["origine_x"], fatigant["origine_y"], c='red', s=100)
ax.scatter(fatigant["destination_x"], fatigant["destination_y"], c='blue', s=100)
plt.show()

In [31]:
fatigant = trajets2[trajets2["closest_node_to_destination"]==482801757].iloc[1]
fatigant
# No path to 482801757  # voyons donc c'est juste à côté..
#nx.shortest_path(G_nobridge,fatigant["closest_node_to_origine"],fatigant["closest_node_to_destination"], weight = "time_minute")


TRIP_ID                                              7961
origine_x                                        -71.3059
origine_y                                         46.8403
destination_x                                    -71.2817
destination_y                                     46.8508
ID_quartier_origine                                    14
ID_quartier_destination                                14
NOM_quartier_origine           Neufchâtel-Est/Lebourgneuf
NOM_quartier_destination       Neufchâtel-Est/Lebourgneuf
origine                           (46.840252, -71.305883)
destination                       (46.850828, -71.281684)
closest_node_to_origine                         307561537
closest_node_to_destination                    2778622453
Name: 7603, dtype: object

In [66]:

pln_nobridge = list(map(lambda x: 1699454165 in x, routesss_nobridge))  # pont pierre laporte vers nord
pls_nobridge = list(map(lambda x: 1699454294 in x, routesss_nobridge)) # pont pierre laporte vers sud
pqc_nobridge = list(map(lambda x: 280618575 in x, routesss_nobridge)) # pont de québec toutes directions
pest_nobridge = list(map(lambda x: 88888888888888888888 in x, routesss_nobridge)) # 3eme lien a l'est

pln_bridge = list(map(lambda x: 1699454165 in x, routesss_bridge))
pls_bridge = list(map(lambda x: 1699454294 in x, routesss_bridge))
pqc_bridge = list(map(lambda x: 280618575 in x, routesss_bridge))
pest_bridge = list(map(lambda x: 88888888888888888888 in x, routesss_bridge))


#print( pln ,pqc ,pest)
#type(pest)

In [77]:
no_route = list(map(lambda x: x == [0], routesss_nobridge))
mean_no_route = pd.Series( (v for v in no_route) ).mean()
mean_no_route

0.0022108172127911566

In [68]:
#output = [[x] for x n p_est]
#type(output)
# list to series
mean_pln_nobridge = pd.Series( (v for v in pln_nobridge) ).mean()
mean_pls_nobridge = pd.Series( (v for v in pls_nobridge) ).mean()
mean_pqc_nobridge = pd.Series( (v for v in pqc_nobridge) ).mean()
mean_pest_nobridge = pd.Series( (v for v in pest_nobridge) ).mean()

mean_pln_bridge = pd.Series( (v for v in pln_bridge) ).mean()
mean_pls_bridge = pd.Series( (v for v in pls_bridge) ).mean()
mean_pqc_bridge = pd.Series( (v for v in pqc_bridge) ).mean()
mean_pest_bridge = pd.Series( (v for v in pest_bridge) ).mean()


print(mean_pln_nobridge, mean_pls_nobridge, mean_pqc_nobridge, mean_pest_nobridge)
     

0.0987761547572049 0.090248716936439 0.0378997236478484 0.0


In [70]:
print(mean_pln_bridge, mean_pls_bridge, mean_pqc_bridge, mean_pest_bridge)

0.08590604026845637 0.07958941966048164 0.037820765890248714 0.023608369522305567


In [71]:
# total sur pont  .. 22% des trajets rtaversent un pont?
mean_pln_nobridge+ mean_pls_nobridge +mean_pqc_nobridge+ mean_pest_nobridge

0.2269245953414923

In [72]:
# total sur pont avec pont a l'est devrait etre le meme.. ok
mean_pln_bridge+ mean_pls_bridge +mean_pqc_bridge+ mean_pest_bridge

0.22692459534149229

In [81]:
type(routesss_bridge)



list

In [ ]:
#nodes
#1699454165 pierre laporte vers nord
#1699454294 pierre laporte vers sud
#
#280618575 pont de québec



In [117]:
type(routesss_bridge[1])


list

In [145]:
# trouverl es edges dans le path
# https://github.com/gboeing/osmnx/blob/master/osmnx/plot.py

edge_nodes = list(zip(routesss_bridge[1][:-1], routesss_bridge[1][1:]))


def path_to_edge(path, G):
    edge_nodes = list(zip(path[:-1], path[1:]))
    return(edge_nodes)
  
z = path_to_edge(routesss_bridge[10], G_nobridge)
z
#edge_paths =  list(map(path_to_edge, 
#                              routesss_bridge))


  # plot the route lines
#    edge_nodes = list(zip(route[:-1], route[1:]))
#    lines = []
#    for u, v in edge_nodes:
#        # if there are parallel edges, select the shortest in length
#        data = min(G.get_edge_data(u, v).values(), key=lambda x: x['length'])
#
#        # if it has a geometry attribute (ie, a list of line segments)
#        if 'geometry' in data and use_geom:
#            # add them to the list of lines to plot
#            xs, ys = data['geometry'].xy
#            lines.append(list(zip(xs, ys)))
#        else:
#            # if it doesn't have a geometry attribute, the edge is a straight
#            # line from node to node
#            x1 = G.nodes[u]['x']
#            y1 = G.nodes[u]['y']
#            x2 = G.nodes[v]['x']
#            y2 = G.nodes[v]['y']
#            line = [(x1, y1), (x2, y2)]
#            lines.append(line)



[(2496754005, 279342131),
 (279342131, 485081090),
 (485081090, 477286351),
 (477286351, 485199696),
 (485199696, 485077686),
 (485077686, 486129907),
 (486129907, 281651289),
 (281651289, 486154415),
 (486154415, 486131509),
 (486131509, 486148924),
 (486148924, 486124734),
 (486124734, 486143660),
 (486143660, 486143680),
 (486143680, 486188891),
 (486188891, 477235566),
 (477235566, 486168893),
 (486168893, 486133353),
 (486133353, 486134859),
 (486134859, 486137723),
 (486137723, 486129540),
 (486129540, 486129566),
 (486129566, 281651417),
 (281651417, 127394961),
 (127394961, 486150405),
 (486150405, 316159197),
 (316159197, 316159166),
 (316159166, 316159146),
 (316159146, 127402055),
 (127402055, 316159131),
 (316159131, 486163677),
 (486163677, 127406335)]

In [141]:
#https://stackoverflow.com/questions/2600191/how-can-i-count-the-occurrences-of-a-list-item

test = [j for i in edge_paths for j in i]
test
from collections import Counter

count_occurrences = Counter(test)
count_occurrences

Counter({(2509788208, 486197960): 14,
         (486197960, 486134608): 14,
         (486134608, 486145115): 14,
         (486145115, 486145053): 14,
         (486145053, 486189946): 14,
         (486189946, 486207318): 14,
         (486207318, 486158405): 16,
         (486158405, 486157164): 16,
         (486157164, 486199819): 16,
         (486199819, 4659607785): 62,
         (4659607785, 2509801016): 64,
         (2509801016, 4584214894): 87,
         (4584214894, 486137790): 87,
         (486137790, 486716147): 87,
         (486716147, 486128986): 87,
         (486128986, 486148096): 90,
         (486148096, 486716157): 88,
         (486716157, 291101916): 236,
         (291101916, 486716210): 142,
         (486716210, 281651356): 361,
         (281651356, 286892203): 324,
         (286892203, 286892129): 433,
         (286892129, 286892166): 360,
         (286892166, 291101977): 368,
         (291101977, 243790303): 643,
         (243790303, 243790308): 585,
         (243790308, 2

19

In [146]:
# add occurrences to graph
for u, v, k, data in G_nobridge.edges(data=True, keys=True):

  data["occurrences"] = count_occurrences[u,v]

#https://geoffboeing.com/2016/11/osmnx-python-street-networks/

ec = ox.get_edge_colors_by_attr(G_nobridge, attr='occurrences')
ox.plot_graph(G_nobridge, edge_color=ec)


ValueError: Bin edges must be unique: array([   0.,    0.,    0.,    0.,    4., 1068.]).
You can drop duplicate edges by setting the 'duplicates' kwarg

Object `G_nobridges.nodes` not found.
